<a href="https://colab.research.google.com/github/mel518/project-4/blob/mel/UFC_Classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from pymongo import MongoClient
import numpy as np

In [ ]:
url = "https://raw.githubusercontent.com/mel518/project-4/mel/data.csv"
data = pd.read_csv(url)
data.head()

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000,0.0,...,0,1,0,0,Orthodox,170.18,177.80,135.0,31.0,27.0
1,Trevin Giles,Roman Dolidze,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500,0.0,...,0,3,0,0,Orthodox,182.88,187.96,185.0,32.0,28.0
2,Tai Tuivasa,Harry Hunsucker,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,NaN,NaN,...,1,3,0,0,Southpaw,187.96,190.50,264.0,32.0,28.0
3,Cheyanne Buys,Montserrat Conejo,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,...,0,0,0,0,Switch,160.02,160.02,115.0,28.0,25.0
4,Marion Reneau,Macy Chiasson,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125,0.0,...,1,2,2,0,Orthodox,167.64,172.72,135.0,29.0,43.0


In [ ]:
# not interested in draws
data['Winner'].value_counts()

Red     3979
Blue    1923
Draw     110
Name: Winner, dtype: int64

In [ ]:
data.nunique()


R_fighter       1514
B_fighter       1987
Referee          204
date             550
location         166
                ... 
R_Height_cms      23
R_Reach_cms       25
R_Weight_lbs      62
B_age             31
R_age             29
Length: 144, dtype: int64

In [ ]:
data.dropna()
df = data[data['Winner'] != 'Draw']
df.drop(columns=['R_fighter', 'B_fighter', 'Referee', 'date', 'location','weight_class','title_bout'], inplace=True)
df = df.fillna(0)
df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Red,0.000,0.0,0.420000,0.49500,0.330,0.36000,0.500,1.0000,0.00,...,0,1,0,0,Orthodox,170.18,177.80,135.0,31.0,27.0
1,Red,0.500,0.0,0.660000,0.30500,0.300,0.50000,1.500,0.0000,0.00,...,0,3,0,0,Orthodox,182.88,187.96,185.0,32.0,28.0
2,Red,0.000,0.0,0.000000,0.00000,0.000,0.00000,0.000,0.0000,0.00,...,1,3,0,0,Southpaw,187.96,190.50,264.0,32.0,28.0
3,Blue,0.000,0.0,0.000000,0.00000,0.000,0.00000,0.000,0.0000,0.00,...,0,0,0,0,Switch,160.02,160.02,115.0,28.0,25.0
4,Blue,0.125,0.0,0.535625,0.57875,0.185,0.16625,0.125,0.1875,0.25,...,1,2,2,0,Orthodox,167.64,172.72,135.0,29.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6007,Red,0.000,0.0,0.000000,0.00000,0.000,0.00000,0.000,0.0000,0.00,...,0,0,0,0,Southpaw,190.50,0.00,260.0,0.0,0.0
6008,Red,0.000,0.0,0.000000,0.00000,0.000,0.00000,0.000,0.0000,0.00,...,0,0,0,0,Southpaw,177.80,0.00,170.0,0.0,0.0
6009,Red,0.000,0.0,0.000000,0.00000,0.000,0.00000,0.000,0.0000,0.00,...,0,0,0,0,Orthodox,182.88,0.00,210.0,0.0,0.0
6010,Red,0.000,0.0,0.000000,0.00000,0.000,0.00000,0.000,0.0000,0.00,...,0,0,0,0,Orthodox,187.96,0.00,225.0,0.0,30.0


In [ ]:
X = df.drop(columns=['Winner'])
Y = df['Winner']
mapping = {'Red': 0, 'Blue': 1}
Y = Y.replace(mapping)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42) 
y_train

2655    1
1043    1
1639    0
5503    0
2773    1
       ..
3850    1
5290    0
5325    0
5489    0
885     1
Name: Winner, Length: 4131, dtype: int64

In [ ]:
X_dummies_train = pd.get_dummies(X_train)
X_dummies_test = pd.get_dummies(X_test)
# y_label_train = LabelEncoder().fit_transform(df['Winner'])
# y_label_test = LabelEncoder().fit_transform(df['Winner'])
print(f"Train: X: {X_dummies_train.shape}, y: {y_train.shape[0]} Test: X {X_dummies_test.shape}, y: {y_test.shape[0]}")

Train: X: (4131, 146), y: 4131 Test: X (1771, 145), y: 1771


In [ ]:
 X_test = X_dummies_test.reindex(columns = X_dummies_train.columns, fill_value=0)
 X_dummies_test = X_test
# y_label_train
# len(y_label_test)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_dummies_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_dummies_train)
X_test_scaled = X_scaler.transform(X_dummies_test)
print(f'{X_train_scaled} {len(X_train_scaled)}')

[[-0.49089993 -0.46382321  0.17048081 ... -0.01556055 -0.50294854
  -0.19140896]
 [-0.00253677 -0.46059446  0.78222892 ... -0.01556055 -0.50294854
  -0.19140896]
 [-0.54113157 -0.46382321  1.86822271 ... -0.01556055 -0.50294854
  -0.19140896]
 ...
 [-0.54113157 -0.46382321 -0.67087491 ... -0.01556055  1.98827498
  -0.19140896]
 [-0.54113157 -0.46382321 -1.56571107 ... -0.01556055 -0.50294854
  -0.19140896]
 [ 1.60208503  1.18929746  1.09642652 ... -0.01556055 -0.50294854
  -0.19140896]] 4131


In [ ]:
X_dummies_train

,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,...,B_Stance_Orthodox,B_Stance_Sideways,B_Stance_Southpaw,B_Stance_Switch,R_Stance_0,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch
2655,0.017578,0.000000,0.388047,0.388633,0.271094,0.264824,0.078125,0.144531,0.125,0.003906,...,0,0,1,0,0,0,1,0,0,0
1043,0.188477,0.000977,0.524775,0.514961,0.004883,0.520283,0.023438,0.500000,0.000,0.250977,...,1,0,0,0,0,0,1,0,0,0
1639,0.000000,0.000000,0.767500,0.527500,0.165000,0.600000,0.000000,0.750000,0.000,0.000000,...,1,0,0,0,0,0,1,0,0,0
5503,0.000000,0.250000,0.332500,0.507500,0.367500,0.580000,0.250000,2.000000,0.250,0.000000,...,0,0,0,1,0,0,1,0,0,0
2773,0.562500,0.519531,0.615703,0.315273,0.148906,0.502930,0.125000,0.031250,0.000,0.000000,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3850,0.750000,0.000000,0.565000,0.335000,0.000000,0.175000,0.750000,0.500000,0.000,0.000000,...,1,0,0,0,0,0,1,0,0,0
5290,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,...,1,0,0,0,0,0,1,0,0,0
5325,0.000000,0.000000,0.200000,1.000000,1.000000,0.000000,0.000000,3.000000,0.000,0.000000,...,0,0,1,0,0,0,0,0,1,0
5489,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,...,1,0,0,0,0,0,1,0,0,0


In [ ]:
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost
from xgboost import XGBClassifier

import keras 
from keras.models import Sequential
from keras.layers import Dense
from keras import layers, models, optimizers
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
seed = 404
np.random.seed(seed)

In [ ]:
from sklearn.linear_model import LinearRegression

classifier = LinearRegression()
classifier 

LinearRegression()

In [ ]:
classifier.fit(X_train_scaled, y_train)
classifier.score(X_train_scaled, y_train)

0.1868734797685152

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train_scaled, y_train)
print(f'{classifier.score(X_train_scaled, y_train)}')
print(f'{classifier.score(X_test_scaled, y_test)}')

x = classifier.predict(X_test_scaled)
redwins = 0
bluewins = 0
for i in x:
  if i == 0:
    redwins = redwins + 1
  else:
    bluewins = bluewins + 1
print(redwins)
print(bluewins)


0.7283950617283951
0.6911349520045172
1381
390


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
import joblib
filename = "fighter_classifier.h5"
joblib.dump(classifier, filename)
loaded_model = joblib.load(filename)
loaded_model

In [ ]:
classifier.get_params().keys()
# Run GridSearchCV on best model, in this case, logistic regression model
from sklearn.model_selection import GridSearchCV

# Define model & parameters
lg = LogisticRegression(fit_intercept=True, random_state =24)
param_grid = {'solver': ['lbfgs', 'newton-cg', 'liblinear'], 
              'penalty': ['l1', 'l2', 'elastic_net', 'none'],}

grid_lg = GridSearchCV(lg, param_grid, verbose=4)

# Fit the model by using the grid search classifier. This will take the SVC model and try each combination of parameters.
grid_lg.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ............penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ............penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ............penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ............penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ............penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END ........penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 2/5] END ........penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 3/5] END ........penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 4/5] END ........penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 5/5] END ........penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 1/5] END ......penalty=l1, solver=liblinear;, score=0.706 total time=   0.7s
[CV 2/5] END ......penalty=l1, solver=liblinear;

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


[CV 2/5] END ..........penalty=l2, solver=lbfgs;, score=0.697 total time=   0.2s
[CV 3/5] END ..........penalty=l2, solver=lbfgs;, score=0.689 total time=   0.1s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


[CV 4/5] END ..........penalty=l2, solver=lbfgs;, score=0.719 total time=   0.2s
[CV 5/5] END ..........penalty=l2, solver=lbfgs;, score=0.697 total time=   0.2s


...

[CV 3/5] END ........penalty=none, solver=lbfgs;, score=0.696 total time=   0.1s
[CV 4/5] END ........penalty=none, solver=lbfgs;, score=0.713 total time=   0.2s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[CV 5/5] END ........penalty=none, solver=lbfgs;, score=0.690 total time=   0.2s
[CV 1/5] END ....penalty=none, solver=newton-cg;, score=0.696 total time=   0.7s
[CV 2/5] END ....penalty=none, solver=newton-cg;, score=0.688 total time=   0.9s
[CV 3/5] END ....penalty=none, solver=newton-cg;, score=0.696 total time=   2.5s
[CV 4/5] END ....penalty=none, solver=newton-cg;, score=0.707 total time=   0.7s
[CV 5/5] END ....penalty=none, solver=newton-cg;, score=0.688 total time=   0.8s
[CV 1/5] END ......penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END ......penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END ......penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END ......penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END ......penalty=none, solver=liblinear;, score=nan total time=   0.0s


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: Solver 

GridSearchCV(estimator=LogisticRegression(random_state=24),
             param_grid={'penalty': ['l1', 'l2', 'elastic_net', 'none'],
                         'solver': ['lbfgs', 'newton-cg', 'liblinear']},
             verbose=4)

In [ ]:
grid_lg.best_params_
grid_lg.best_score_
grid_lg.score(X_test_scaled, y_test)
#better without hypertuning

0.6843591191417279

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
classifier = ExtraTreesClassifier(n_estimators=1000)
classifier.fit(X_train_scaled, y_train)
print(f'{classifier.score(X_train_scaled, y_train)}')
print(f'{classifier.score(X_test_scaled, y_test)}')

x = classifier.predict(X_test_scaled)
redwins = 0
bluewins = 0
for i in x:
  if i == 0:
    redwins = redwins + 1
  else:
    bluewins = bluewins + 1
print(redwins)
print(bluewins)

1.0
0.6866177300959909
1605
166


In [ ]:
classifier = RandomForestClassifier()
classifier.fit(X_train_scaled, y_train)
print(f'{classifier.score(X_train_scaled, y_train)}')
print(f'{classifier.score(X_test_scaled, y_test)}')
x = classifier.predict(X_test_scaled)
redwins = 0
bluewins = 0
for i in x:
  if i == 0:
    redwins = redwins + 1
  else:
    bluewins = bluewins + 1
print(redwins)
print(bluewins)

1.0
0.6826651609260305
1568
203


In [ ]:
df.columns.values

array(['Winner', 'B_avg_KD', 'B_avg_opp_KD', 'B_avg_SIG_STR_pct',
       'B_avg_opp_SIG_STR_pct', 'B_avg_TD_pct', 'B_avg_opp_TD_pct',
       'B_avg_SUB_ATT', 'B_avg_opp_SUB_ATT', 'B_avg_REV', 'B_avg_opp_REV',
       'B_avg_SIG_STR_att', 'B_avg_SIG_STR_landed',
       'B_avg_opp_SIG_STR_att', 'B_avg_opp_SIG_STR_landed',
       'B_avg_TOTAL_STR_att', 'B_avg_TOTAL_STR_landed',
       'B_avg_opp_TOTAL_STR_att', 'B_avg_opp_TOTAL_STR_landed',
       'B_avg_TD_att', 'B_avg_TD_landed', 'B_avg_opp_TD_att',
       'B_avg_opp_TD_landed', 'B_avg_HEAD_att', 'B_avg_HEAD_landed',
       'B_avg_opp_HEAD_att', 'B_avg_opp_HEAD_landed', 'B_avg_BODY_att',
       'B_avg_BODY_landed', 'B_avg_opp_BODY_att', 'B_avg_opp_BODY_landed',
       'B_avg_LEG_att', 'B_avg_LEG_landed', 'B_avg_opp_LEG_att',
       'B_avg_opp_LEG_landed', 'B_avg_DISTANCE_att',
       'B_avg_DISTANCE_landed', 'B_avg_opp_DISTANCE_att',
       'B_avg_opp_DISTANCE_landed', 'B_avg_CLINCH_att',
       'B_avg_CLINCH_landed', 'B_avg_opp_CLINCH_a